In [1]:
import hopsworks

project = hopsworks.login()  # insert API Key from https://app.hopsworks.ai

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://hopsworks0.logicalclocks.com/p/119


## Requirements

Install libraries:

* **tensorflow** (version 2.11) [already installed]
* **tensorflow-addons** (version 0.20.0)
* **tensorflow-recommenders** (version 0.7.3)

In [22]:
!pip install --quiet tensorflow-addons==0.20.0
!pip install --quiet tensorflow-recommenders==0.7.3

## Train Retrieval Model

In this notebook, we will train a retrieval model that will be able to quickly generate a small subset of candidate items from a large collection of items. Our model will be based on the two-tower architecture, which embeds queries and candidates (keys) into a shared low-dimensional vector space. Here, a query consists of features of a customer and a transaction (e.g. timestamp of the purchase), whereas a candidate consists of features of a particular item. All queries will have a user ID and all candidates will have an item ID, and the model will be trained such that the embedding of a user will be close to all the embeddings of items the user has previously bought.

After training the model we will save and upload its components to the Hopsworks Model Registry.

### Data

Let's go ahead and load the data.

In [8]:
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [9]:
feature_view = fs.get_feature_view("retrieval", version=1)

train_df, val_df, test_df, y_train, y_val, y_test = feature_view.get_train_validation_test_split(training_dataset_version=1)

We will train our retrieval model with a subset of features.

For the query embedding we will use:
- `customer_id`: ID of the customer.
- `age`: age of the customer at the time of purchase.
- `month_sin`, `month_cos`: time of year the purchase was made.

For the candidate embedding we will use:
- `article_id`: ID of the item.
- `garment_group_name`: type of garment.
- `index_group_name`: menswear/ladieswear etc.

In [10]:
train_df["article_id"] = train_df["article_id"].astype(str)  # to be removed
val_df["article_id"] = val_df["article_id"].astype(str)  # to be removed

In [11]:
import tensorflow as tf

query_features = ["customer_id", "age", "month_sin", "month_cos"]
candidate_features = ["article_id", "garment_group_name", "index_group_name"]

def df_to_ds(df):
    return tf.data.Dataset.from_tensor_slices({col : df[col] for col in df})

BATCH_SIZE = 2048
train_ds = df_to_ds(train_df).batch(BATCH_SIZE).cache().shuffle(BATCH_SIZE*10)
val_ds = df_to_ds(val_df).batch(BATCH_SIZE).cache()

We will need a list of user and item IDs when we initialize our embeddings.

In [12]:
user_id_list = train_df["customer_id"].unique().tolist()
item_id_list = train_df["article_id"].unique().tolist()

garment_group_list = train_df["garment_group_name"].unique().tolist()
index_group_list = train_df["index_group_name"].unique().tolist()

print(f"Number of transactions: {len(train_df):,}")
print(f"Number of users: {len(user_id_list):,}")
print(f"Number of items: {len(item_id_list):,}")
print(garment_group_list)

Number of transactions: 421,427
Number of users: 24,189
Number of items: 58,345
['Trousers Denim', 'Outdoor', 'Socks and Tights', 'Trousers', 'Jersey Fancy', 'Jersey Basic', 'Blouses', 'Unknown', 'Accessories', 'Under-, Nightwear', 'Knitwear', 'Dresses Ladies', 'Skirts', 'Shorts', 'Swimwear', 'Shoes', 'Dressed', 'Special Offers', 'Woven/Jersey/Knitted mix Baby', 'Shirts', 'Dresses/Skirts girls']


### Two Tower Model

The two tower model consist of two models:
- Query model: generates a query representation given user and transaction features.
- Candidate model: generates an item representation given item features.

Both models produce embeddings that live in the same embedding space. We let this space be low-dimensional to prevent overfitting on the training data. (Otherwise, the model might simply memorize previous purchases, which makes it recommend items customers already have bought.)

In [13]:
EMB_DIM = 16

We start with creating the query model.

In [14]:
from tensorflow.keras.layers.experimental.preprocessing import StringLookup, Normalization

In [15]:
class QueryTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.user_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=user_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # We add an additional embedding to account for unknown tokens.
                len(user_id_list) + 1,
                EMB_DIM
            )
        ])

        self.normalized_age = Normalization(axis=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(EMB_DIM, activation="relu"),
            tf.keras.layers.Dense(EMB_DIM)
        ])

    def call(self, inputs):
        concatenated_inputs = tf.concat([
            self.user_embedding(inputs["customer_id"]),
            tf.reshape(self.normalized_age(inputs["age"]), (-1,1)),
            tf.reshape(inputs["month_sin"], (-1,1)),
            tf.reshape(inputs["month_cos"], (-1,1))
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


query_model = QueryTower()

query_model.normalized_age.adapt(train_ds.map(lambda x : x["age"]))

# Initialize model with inputs.
query_df = train_df[query_features]
query_ds = df_to_ds(query_df).batch(1)
query_model(next(iter(query_ds)))

2023-07-10 19:04:20,104 WARNING: From /srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/tensorflow/python/autograph/pyct/static_analysis/liveness.py:83: Analyzer.lamba_check (from tensorflow.python.autograph.pyct.static_analysis.liveness) is deprecated and will be removed after 2023-09-23.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


<tf.Tensor: shape=(1, 16), dtype=float32, numpy=
array([[-0.11404308,  0.05979271, -0.20524183,  0.02402569, -0.18664078,
        -0.05092149, -0.00738733, -0.05473415, -0.01609503, -0.00702365,
        -0.14195976, -0.0490661 ,  0.06219643, -0.06058411, -0.0356596 ,
        -0.0484673 ]], dtype=float32)>

The candidate model is very similar to the query model. A difference is that it has two categorical features as input, which we one-hot encode.

In [16]:
class ItemTower(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.item_embedding = tf.keras.Sequential([
            StringLookup(
                vocabulary=item_id_list,
                mask_token=None
            ),
            tf.keras.layers.Embedding(
                # We add an additional embedding to account for unknown tokens.
                len(item_id_list) + 1,
                EMB_DIM
            )
        ])

        self.garment_group_tokenizer = StringLookup(vocabulary=garment_group_list, mask_token=None)
        self.index_group_tokenizer = StringLookup(vocabulary=index_group_list, mask_token=None)

        self.fnn = tf.keras.Sequential([
            tf.keras.layers.Dense(EMB_DIM, activation="relu"),
            tf.keras.layers.Dense(EMB_DIM)
        ])

    def call(self, inputs):
        garment_group_embedding = tf.one_hot(
            self.garment_group_tokenizer(inputs["garment_group_name"]),
            len(garment_group_list)
        )

        index_group_embedding = tf.one_hot(
            self.index_group_tokenizer(inputs["index_group_name"]),
            len(index_group_list)
        )

        concatenated_inputs = tf.concat([
            self.item_embedding(inputs["article_id"]),
            garment_group_embedding,
            index_group_embedding
        ], axis=1)

        outputs = self.fnn(concatenated_inputs)

        return outputs


item_model = ItemTower()

We will evaluate the two tower model using the *top-100 accuracy*. That is, for each transaction in the validation data we will generate the associated query embedding and retrieve the set of the 100 items that are closest to this query in the embedding space. The top-100 accuracy measures how often the item that was actually bought is part of this subset. To evaluate this, we create a dataset of all unique items in the training data.

In [17]:
item_df = train_df[candidate_features]
item_df.drop_duplicates(subset="article_id", inplace=True)
item_ds = df_to_ds(item_df)

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


With this in place, we can finally create our two tower model.

In [20]:
import tensorflow_recommenders as tfrs

class TwoTowerModel(tf.keras.Model):
    def __init__(self, query_model, item_model):
        super().__init__()
        self.query_model = query_model
        self.item_model = item_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=item_ds.batch(BATCH_SIZE).map(self.item_model)
            )
        )

    def train_step(self, batch) -> tf.Tensor:
        # Set up a gradient tape to record gradients.
        with tf.GradientTape() as tape:

            # Loss computation.
            user_embeddings = self.query_model(batch)
            item_embeddings = self.item_model(batch)
            loss = self.task(user_embeddings, item_embeddings,
                             compute_metrics=False)

            # Handle regularization losses as well.
            regularization_loss = sum(self.losses)

            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        metrics = {
            "loss": loss,
            "regularization_loss": regularization_loss,
            "total_loss": total_loss
        }

        return metrics

    def test_step(self, batch) -> tf.Tensor:
        # Loss computation.
        user_embeddings = self.query_model(batch)
        item_embeddings = self.item_model(batch)

        loss = self.task(user_embeddings, item_embeddings,
                         compute_metrics=False)

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss

        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics


#### Model Training

We'll train our model using the AdamW optimizer, which applies weight regularization during training.

In [23]:
import tensorflow_addons as tfa

model = TwoTowerModel(query_model, item_model)
optimizer = tfa.optimizers.AdamW(0.001, learning_rate=0.01)
model.compile(optimizer=optimizer)


TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 



In [24]:
model.fit(train_ds, validation_data=val_ds, epochs=5)

Epoch 1/5
206/206 [==============================] - 46s 202ms/step - loss: 14969.3742 - regularization_loss: 0.0000e+00 - total_loss: 14969.3742 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - val_loss: 10981.9775 - val_regularization_loss: 0.0000e+00 - val_total_loss: 10981.9775
Epoch 2/5
206/206 [==============================] - 39s 188ms/step - loss: 14272.7879 - regularization_loss: 0.0000e+00 - total_loss: 14272.7879 - val_factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - val_factorized_top_k/top_100_categorical_accuracy: 0

### Upload Model to Model Registry

One of the features in Hopsworks is the model registry. This is where we can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

Let's connect to the model registry using the [HSML library](https://docs.hopsworks.ai/machine-learning-api/latest) from Hopsworks.

In [25]:
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [27]:
class QueryModelModule(tf.Module):
    def __init__(self, query_model):
        self.query_model = query_model

    @tf.function()
    def compute_emb(self, instances):
        query_emb = self.query_model(instances)
        return {"customer_id": instances["customer_id"],
                "month_sin": instances["month_sin"],
                "month_cos": instances["month_cos"],
                "query_emb": query_emb}

# wrap query_model:   query_model -> query_model_module
query_model = QueryModelModule(model.query_model)

In [28]:
instances_spec={
    'customer_id': tf.TensorSpec(shape=(None,), dtype=tf.string, name='customer_id'),
    'month_sin': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_sin'),
    'month_cos': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='month_cos'),
    'age': tf.TensorSpec(shape=(None,), dtype=tf.float64, name='age')
}
signatures = query_model.compute_emb.get_concrete_function(instances_spec)

tf.saved_model.save(query_model, "query_model", signatures=signatures)

2023-07-10 19:28:28,490 INFO: Assets written to: query_model/assets


First, we need to save our models locally.

In [29]:
tf.saved_model.save(model.item_model, "candidate_model")

2023-07-10 19:28:44,422 INFO: Assets written to: candidate_model/assets


Each model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model. A schema can either be manually specified or inferred from data.

In [30]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# Infer input schema from data.
query_model_input_schema = Schema(query_df)

# Manually specify output schema.
query_model_output_schema = Schema([{
    "name": "query_embedding",
    "type": "float32",
    "shape": [EMB_DIM]
}])

query_model_schema = ModelSchema(
    input_schema=query_model_input_schema,
    output_schema=query_model_output_schema
)

query_model_schema.to_dict()

{'input_schema': {'columnar_schema': [{'name': 'customer_id',
    'type': 'object'},
   {'name': 'age', 'type': 'float64'},
   {'name': 'month_sin', 'type': 'float64'},
   {'name': 'month_cos', 'type': 'float64'}]},
 'output_schema': {'tensor_schema': [{'name': 'query_embedding',
    'shape': '[16]',
    'type': 'float32'}]}}

With the schema in place, we can finally register our model.

In [31]:
query_example = query_df.sample().to_dict("records")

mr_query_model = mr.tensorflow.create_model(
    name="query_model",
    description="Model that generates query embeddings from user and transaction features",
    input_example=query_example,
    model_schema=query_model_schema
)

mr_query_model.save("query_model")

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://hopsworks0.logicalclocks.com/p/119/models/query_model/1


Model(name: 'query_model', version: 1)

Here we have also saved an input example from the training data, which can be helpful for test purposes.

Let's repeat the process with the candidate model.

In [32]:
candidate_model_input_schema = Schema(item_df)

candidate_model_output_schema = Schema([{
    "name": "candidate_embedding",
    "type": "float32",
    "shape": [EMB_DIM]}
])

candidate_model_schema = ModelSchema(
    input_schema=candidate_model_input_schema,
    output_schema=candidate_model_output_schema
)

candidate_example = item_df.sample().to_dict("records")

mr_candidate_model = mr.tensorflow.create_model(
    name="candidate_model",
    description="Model that generates candidate embeddings from item features",
    input_example=candidate_example,
    model_schema=candidate_model_schema
)

mr_candidate_model.save("candidate_model")

  0%|          | 0/6 [00:00<?, ?it/s]

Model created, explore it at https://hopsworks0.logicalclocks.com/p/119/models/candidate_model/1


Model(name: 'candidate_model', version: 1)

### Next Steps

Retrieving the top-k closest candidate embeddings in a brute-force way (computing the distances between the query embedding and all candidate embeddings) is too expensive in a practical setting. In the next notebook, we will index the item embeddings using OpenSearch, which will allow us to retrieve candidates with very low latency.